In [ ]:
from dataset_factory import TextDataset, getDataloaders
import seaborn as sns # For class distribution visualization
import matplotlib.pyplot as plt
from file_utils import *
from transformers import BertTokenizer
import torch
from torch.utils.data import random_split # For custom data-sets
import pandas as pd

In [ ]:
# Uncomment to use this version of TextDataset

# class TextDataset(Dataset):
#     def __init__(self, csv_file, max_length):
#         self.data = pd.read_csv(csv_file)
#         self.data = self.data[self.data['lang_abv'] == 'en'] # Drop non-english rows 
#         self.max_length = max_length
#         self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#     def __len__(self):
#         return len(self.data)
    
#     # Geeling: This is the string to word_ids using PA4 vocab, but if pretrained BERT tokenizer works, let's use that instead
# #     def __convert_string_to_word_ids(self, string):
# #         # Convert a string to a Tensor of word ids.
        
# #         vocab = self.vocab
# #         tokens = nltk.tokenize.word_tokenize(str(string).lower())
# #         word_ids = [vocab('<start>')]
# #         word_ids.extend([vocab(token) for token in tokens])
# #         word_ids.append(vocab('<end>'))
# #         word_ids_tensor = torch.Tensor(word_ids)
# #         return word_ids_tensor
    
#     def _tokenize(self, x):
#         # This will truncate or pad to self.max_length, such that the return array is self.max_length in length. 
#         # [UNK] token is 100
#         # start token is 101
#         # [SEP] token is 102
#         # [PAD] token is 0
#         # See more here: https://huggingface.co/transformers/model_doc/bert.html?highlight=berttokenizer#berttokenizer 
#         return self.tokenizer(
#                 x,
#                 max_length=self.max_length, 
#                 truncation=True, 
#                 padding='max_length')
    
#     def __getitem__(self, idx):
        
#         prem = self.data.iloc[idx]['premise']
#         hypo = self.data.iloc[idx]['hypothesis']
        
#         # Tokenize with BERT 
#         # TODO: make use of 'token_type_ids' and 'attention_mask' (attention will disregard the PAD tokens)
#         # Currently only uses 'input_ids' which is a tokenized representation similar to 
#         premise = self._tokenize(prem)['input_ids']
#         hypothesis = self._tokenize(hypo)['input_ids']
        
#         # Convert strings to word ids. NOT USED
#         # premise = self.__convert_string_to_word_ids(prem)
#         # hypothesis = self.__convert_string_to_word_ids(hypo)
        
#         return torch.Tensor(premise), torch.Tensor(hypothesis), self.data.iloc[idx]['label']

In [ ]:
# Uncomment to use this version of getDataloaders()

# def getDataloaders(csv_file, config_data):
# #     def collate_fn(data):
# #         """Creates mini-batch tensors from the list of tuples (image, caption)
# #         by padding the captions to make them of equal length.
# #         We can not use default collate_fn because variable length tensors can't be stacked vertically.
# #         We need to pad the captions to make them of equal length so that they can be stacked for creating a mini-batch.
# #         Args:
# #             data: list of tuple (premises, hypothesis, label).
# #                 - premises: torch tensor of shape (?); variable length.
# #                 - hypothesis: torch tensor of shape (?); variable length.
# #                 - label: 0, 1, or 2
# #         Returns:
# #             padded_prem: torch tensor of shape (batch_size, prem_padded_length).
# #             padded_hypo: torch tensor of shape (batch_size, hypo_padded_length).
# #             labels: list; original labels.
# #         """
# #         # Sort a data list by caption length (descending order).
# #         # Geeling: don't think this is necessary, but was in the original collate_fn for some reason
# #         # data.sort(key=lambda x: len(x[1]), reverse=True)

# #         # Unpack original data
# #         premises, hypothesis, labels = zip(*data)

# #         # Pad premises 
# #         lengths = [len(prem) for prem in premises]
# #         padded_prem = torch.zeros(len(premises), max(lengths)).long()

# #         for i, prem in enumerate(premises):
# #             end = lengths[i]
# #             padded_prem[i, :end] = prem[:end]

# #         # Pad hypothesis
# #         lengths = [len(hyp) for hyp in hypothesis]
# #         padded_hypo = torch.zeros(len(hypothesis), max(lengths)).long()

# #         for i, hypo in enumerate(hypothesis):
# #             end = lengths[i]
# #             padded_hypo[i, :end] = hypo[:end]

# #         return padded_prem, padded_hypo, labels

#     all_data = TextDataset(csv_file, config_data['generation']['max_length'])
#     num_train = int(len(all_data) * 0.7)
#     num_val = int(len(all_data) * 0.2)
#     num_test = int(len(all_data) * 0.1)
    
#     torch.manual_seed(torch.initial_seed())
#     train_dataset, val_dataset, test_dataset = random_split(all_data, (num_train, num_val, num_test))

#     train_dataloader = DataLoader(dataset=train_dataset,
#                       batch_size=config_data['dataset']['batch_size'],
#                       shuffle=True,
#                       num_workers=config_data['dataset']['num_workers'],
#                       #collate_fn=collate_fn,
#                       pin_memory=True) # Pin memory makes it faster to move from CPU to GPU (may look into loading dataset to GPU directly tho)
#     val_dataloader = DataLoader(dataset=val_dataset,
#                       batch_size=config_data['dataset']['batch_size'],
#                       shuffle=True,
#                       num_workers=config_data['dataset']['num_workers'],
#                       #collate_fn=collate_fn,
#                       pin_memory=True)
#     test_dataloader = DataLoader(dataset=test_dataset,
#                       batch_size=config_data['dataset']['batch_size'],
#                       shuffle=False,
#                       num_workers=config_data['dataset']['num_workers'],
#                       #collate_fn=collate_fn,
#                       pin_memory=True)
#     return train_dataloader, val_dataloader, test_dataloader
    

## Dataloader playground

In [ ]:
# Load config
config = read_file("./config.json")

In [ ]:
# See that config is loaded properly
config

In [ ]:
# Get the dataloaders
train_dl, val_dl, test_dl = getDataloaders('./data/train.csv', config)

In [ ]:
# Print out some details of the first batch in test dataloader as a test
for i, (prem, hypo, label) in enumerate(test_dl):
    if i % 500 == 0: 
        inputs = prem.to('cpu')
        print(inputs.shape) # n_batch_elems, max_length
        print(hypo[0]) # First hypothesis
        print(len(label)) # number of labels

## Tokenizer playground

In [ ]:
# Load pretrained tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# BERT has 2x the vocab size as the COCO dataset
tokenizer.vocab_size 

In [ ]:
# Or put it together and include padding requirements
inputs = tokenizer("Hello, my dog is cute [PAD] [PAD] [SEP] he hehe [PAD][PAD][PAD] ", max_length=25, 
                truncation = True,
                padding='max_length')
inputs # This will be a dictionary with 'input_ids', 'token_type_ids', 'attention_mask'

In [ ]:
# Tokenizing can happen in two steps
inputs = tokenizer.tokenize("Hello, my dog is cute [PAD] [PAD] [SEP] he hehe [PAD][PAD][PAD] ")
for i in tokenizer.convert_tokens_to_ids(inputs):
    print(tokenizer._convert_id_to_token(i))

## Dataset playground

In [ ]:
# Load all the training data as all our data (we technically only have train.csv to work with since it has labels)
# TextDataset will automatically only select 'en' English rows
csv_file = './data/train.csv'
all_data = TextDataset(csv_file, 20)
len(all_data)

In [ ]:
# Split train, val, test
num_train = int(len(all_data) * 0.7)
num_val = int(len(all_data) * 0.2)
num_test = int(len(all_data) * 0.1)

# Make sure to check that your split produces integer vals that add up to the total number in all_data
print(num_train+num_val+num_test)

# Random split
torch.manual_seed(torch.initial_seed())
train_dataset, val_dataset, test_dataset = random_split(all_data, (num_train, num_val, num_test))

In [ ]:
# Count the number of each class we have in each dataset
def tally_classes(dataset):
    # Assumes last element is the class label: premise, hypothesis, label
    class_count = {}
    for d in dataset: 
        label = d[2]
        if label not in class_count: 
            class_count[label] = 0
        class_count[label] += 1 
        
    return class_count

In [ ]:
# Check distribution of random split

sns.barplot(data = pd.DataFrame.from_dict([tally_classes(train_dataset)]).melt(), x="variable", y="value", hue="variable").set_title('Hypothesis Type Distribution')
plt.show()

sns.barplot(data = pd.DataFrame.from_dict([tally_classes(val_dataset)]).melt(), x ="variable", y="value", hue="variable").set_title('Hypothesis Type Distribution')
plt.show()

sns.barplot(data = pd.DataFrame.from_dict([tally_classes(test_dataset)]).melt(), x="variable", y="value", hue="variable").set_title('Hypothesis Type Distribution')
plt.show()

